### Impoting Necessary and Useful Libraries

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

### Scraping data from Wikipedia page into a DataFrame

In [ ]:
pd.set_option('display.max_colwidth', -1)
df = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M",header=0)[0]
df

### Removing cells with a borough that is "Not assigned"

In [ ]:
df = df[df.Borough != "Not assigned"].reset_index(drop=True)
df.head()

### Combining the neighbourhoods

In [ ]:
df = df.groupby(["Postcode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df

### For Neighbourhood = "Not assigned", make its value the same as Borough

In [ ]:
for x,y in df.iterrows():
    if y["Neighbourhood"] == "Not assigned":
        y["Neighbourhood"] = y["Borough"]
        
df


### Printing the shape of final dataframe

In [ ]:
df.shape

### Loading the Latitude and Longitude Data

In [ ]:
df1 = pd.read_csv(("https://cocl.us/Geospatial_data"))
df1.head()

### Preparing the DataFrames for merging them

In [ ]:
df1.rename(columns={"Postal Code": "Postcode"}, inplace=True)
df1.head()

### Creating the final DataFrame

In [ ]:
final = df.merge(df1, on="Postcode", how="left")
final.head()

### Finding neighbourhoods of toronto

In [ ]:
toronto = final[final['Borough'].str.contains("Toronto", case=False)]
toronto.head()

### Plotting the data for analysis

In [ ]:
import folium

In [ ]:
map_toronto = folium.Map(location=[43.653963, -79.387207], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(toronto['Latitude'], toronto['Longitude'], toronto['Borough'], toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto